Topic --> Generate Outline using LLM --> Generate a blog using the outline using LLM  --> END

In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [5]:
def create_outline(state: BlogState) -> BlogState:

    #fetch title
    title = state['title']

    # Call LLM to generate outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    #update
    state['outline'] = outline

    return state

In [7]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the following outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [10]:
graph = StateGraph(BlogState)

#nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

#edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()


In [11]:
initial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(initial_state)

In [12]:
final_state

{'title': 'Rise of AI in India',
 'outline': "# Outline for Blog: The Rise of AI in India\n\n## Introduction\n- **Hook**: A compelling anecdote or statistic that highlights the rapid growth of AI technology globally, with a specific focus on India.\n- **Purpose**: Discuss the increasing influence of artificial intelligence in India and its implications for various sectors.\n- **Scope**: Briefly mention the areas that will be covered, including technology, economy, education, and societal impact.\n\n## I. Overview of Artificial Intelligence\n- **Definition of AI**: What constitutes AI? Different types of AI (Narrow AI, General AI, etc.).\n- **Global Context**: Brief overview of the global AI landscape and advancements, leading into the focus on India.\n\n## II. Historical Context of AI in India\n- **Early Adoption**: Discuss the initial stages of AI research and development in India, including academic contributions.\n- **Milestones**: Key events that marked significant progress in AI t

In [14]:
final_state['content']

"# The Rise of AI in India\n\n## Introduction\nThe surge of artificial intelligence (AI) globally is nothing short of astonishing. According to a report, the global AI market is expected to reach a staggering $390 billion by 2025, with Asia-Pacific, particularly India, playing a key role in this revolution. From chatbots answering queries to autonomous vehicles navigating crowded streets, AI is poised to reshape our world. In India, this technological wave is not just a trend; it’s a transformation that is influencing various sectors, enhancing productivity, and forging new pathways in innovation.\n\nIn this blog, we will explore the growing influence of AI in India, discussing its implications across technology, the economy, education, and society while examining the unique opportunities and challenges faced in this context.\n\n## I. Overview of Artificial Intelligence\nArtificial Intelligence, in its simplest form, refers to the capability of a machine to mimic human cognitive functi

If you did the same in langchain, you would have only got the content and not the outline. As you use state in LangGraph, it acts like a memory